<a href="https://colab.research.google.com/github/daewpark/web_scrapping/blob/main/eunpyung_scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Package 설치

In [2]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

     |████████████████████████████████| 911kB 4.1MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/d

### Pactkage 호출 및 로드

In [3]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
import time

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--window-size=1920x1080')
driver =webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.


In [4]:
import urllib.request
from bs4 import BeautifulSoup
import re

### Selenium WebDriver 구동 및 검색결과 Parsing

In [5]:
url = "https://www.ep.go.kr/CmsWeb/viewPage.req?idx=PG0000001180"
driver.get(url)
time.sleep(1)

In [6]:
# 로드 된 웹페이지에서 검색창의 Name 을 찾아서 검색어 입력
search = driver.find_element_by_name("searTot")
search.send_keys("원아모집") # headless 모드에서 chrome_options.add_argument('--window-size=1920x1080') 옵션이 없으면 에러 발생하므로 옵션을 반드시 넣을 것
search.submit()


In [12]:
# 검색결과 파싱
driver.switch_to.window(driver.window_handles[-1]) #검색결과가 다른 탭에서 열리므로 새롭게 열린 탭으로 이동
search_html = driver.page_source
soup = BeautifulSoup(search_html,'html.parser')
title = soup.find_all(class_='btnS_nw')

[<a class="btnS_nw" href="http://www.ep.go.kr/CmsWeb/viewPage.req?idx=PG0000001131&amp;boardId=BO0000000087&amp;CP0000000002_BO0000000087_Action=boardView&amp;CP0000000002_BO0000000087_ViewName=board/BoardView&amp;boardDataId=211932" target="_blank" title="구립 물빛캐슬어린이집 원아모집 안내새창열기"><span>새창열기</span></a>, <a class="btnS_nw" href="http://www.ep.go.kr/CmsWeb/viewPage.req?idx=PG0000001131&amp;boardId=BO0000000087&amp;CP0000000002_BO0000000087_Action=boardView&amp;CP0000000002_BO0000000087_ViewName=board/BoardView&amp;boardDataId=211446" target="_blank" title="『구립 응암행복어린이집』원아모집 안내새창열기"><span>새창열기</span></a>, <a class="btnS_nw" href="http://www.ep.go.kr/CmsWeb/viewPage.req?idx=PG0000001131&amp;boardId=BO0000000087&amp;CP0000000002_BO0000000087_Action=boardView&amp;CP0000000002_BO0000000087_ViewName=board/BoardView&amp;boardDataId=211445" target="_blank" title="『구립 응암샛별어린이집』원아모집 안내새창열기"><span>새창열기</span></a>, <a class="btnS_nw" href="http://www.ep.go.kr/CmsWeb/viewPage.req?idx=PG0000001131&amp

### Pandas DataFrame 으로 결과저장

In [13]:
import pandas as pd

result_list = []

for i in title :
    ititle = i.attrs['title']
    ilink = i.attrs['href']
    result_list.append([ititle,ilink])

df = pd.DataFrame(result_list, columns = ["title","link"])
print(df)

                      title                                               link
0   구립 물빛캐슬어린이집 원아모집 안내새창열기  http://www.ep.go.kr/CmsWeb/viewPage.req?idx=PG...
1  『구립 응암행복어린이집』원아모집 안내새창열기  http://www.ep.go.kr/CmsWeb/viewPage.req?idx=PG...
2  『구립 응암샛별어린이집』원아모집 안내새창열기  http://www.ep.go.kr/CmsWeb/viewPage.req?idx=PG...
3    구립 이든어린이집 원아 모집 안내새창열기  http://www.ep.go.kr/CmsWeb/viewPage.req?idx=PG...
